<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/EVI_and_Calculations_Example_PERU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
ee.Authenticate()

ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=whEZeWPqJh3ylkVphvXxpt4dL2S9Cp0DTRqFWYVWxns&tc=7i8ccUAlQkkFflmMy8NRdfQITfgaAFtCnujJmV_-8Yg&cc=QYHKiKn-Nfp2E5v7PsQsleEU-VqoEW8VAlZlSGZIa5I

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXn8C7BdxtlBYgHvX7U984DyghHBrUcygdoaqGYzyxYFqx__DwySyW4

Successfully saved authorization token.


Enhanced Vegetation Index Worksheet
- Advantage here is it adds blueband to correct for aerosol influence.
- EVI is more responsive to canopy structural variations
- In the presense of snow, NDVI decreases and EVI increases

In [73]:
#Country Boundary
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
Peru = countries.filter(ee.Filter.eq('country_na', 'Peru'))

In [74]:
#Landsat Data
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
.filterDate('2017-01-01', '2017-12-31')\
.filterBounds(Peru)\
.median()

#Computing EVI using expression
evi = landsat.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
     {  'NIR': landsat.select('B5'),  # NIR band
        'RED': landsat.select('B4'),  # Red band
        'BLUE': landsat.select('B2')  # Blue band
})

In [75]:
#Colour Palette for the map
EVI_Palette = 'e5f5f9', '99d8c9', '2ca25f'

In [76]:
#Imported go-to Folium imported and code tree
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[-9.2, -75], zoom_start = 6)

# Display
my_map.add_ee_layer(evi.clip(Peru), {'min': -0.4, 'max': 0.5, 'palette': EVI_Palette}, 'EVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)